In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.

In [2]:
# import env file for hostname, username, password, and db_name
from env import host, user, password, db_name

In [3]:
# Pass env file authentication to container 'url'
url = f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

In [4]:
# define sql search for all records from all tables in Zillow database
sql = """
SELECT prop.*,
       pred.logerror, 
       pred.transactiondate, 
       air.airconditioningdesc, 
       arch.architecturalstyledesc, 
       build.buildingclassdesc, 
       heat.heatingorsystemdesc, 
       landuse.propertylandusedesc, 
       story.storydesc, 
       construct.typeconstructiondesc 
FROM   properties_2017 prop  
JOIN (SELECT parcelid,
				  logerror,
				  Max(transactiondate) transactiondate 
		   FROM   predictions_2017 
		   GROUP  BY parcelid, logerror) pred
	   USING (parcelid)
JOIN propertylandusetype USING (propertylandusetypeid)
LEFT JOIN airconditioningtype air USING (airconditioningtypeid) 
LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid) 
LEFT JOIN buildingclasstype build USING (buildingclasstypeid) 
LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid) 
LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid) 
LEFT JOIN storytype story USING (storytypeid) 
LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid) 
WHERE  prop.latitude IS NOT NULL 
AND prop.longitude IS NOT NULL
AND transactiondate < '2018-01-01' 
AND propertylandusetypeid = 261 
"""

In [96]:
# load zillow data from saved csv or pull from sql server and save to csv
import os
file = 'zillow_data.csv'
if os.path.isfile(file):
    df = pd.read_csv(file, index_col=0)
else:
    df = pd.read_sql(sql,url)
    df.to_csv(file)

In [6]:
df.head()

,parcelid,typeconstructiontypeid,storytypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,propertylandusetypeid,id,basementsqft,...,censustractandblock,propertylandusedesc,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,NaN,NaN,NaN,NaN,261.0,1727539,NaN,...,6.059063e+13,Single Family Residential,0.025595,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN
1,17052889,NaN,NaN,NaN,NaN,NaN,NaN,261.0,1387261,NaN,...,6.111001e+13,Single Family Residential,0.055619,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN
2,14186244,NaN,NaN,NaN,NaN,NaN,NaN,261.0,11677,NaN,...,6.059022e+13,Single Family Residential,0.005383,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN
3,12177905,NaN,NaN,2.0,NaN,NaN,NaN,261.0,2288172,NaN,...,6.037300e+13,Single Family Residential,-0.103410,2017-01-01,NaN,NaN,NaN,Central,NaN,NaN
4,12095076,NaN,NaN,2.0,NaN,NaN,1.0,261.0,781532,NaN,...,6.037461e+13,Single Family Residential,-0.001011,2017-01-01,Central,NaN,NaN,Central,NaN,NaN


In [7]:
# df shape
df.shape

(52438, 68)

In [8]:
df.parcelid.duplicated().sum()

118

In [9]:
# Sort df by parcelid and then by transaction date to group by parcelid and to make sure transaction date is last
df = df.sort_values(by=['parcelid','transactiondate'])
df.head()

,parcelid,typeconstructiontypeid,storytypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,propertylandusetypeid,id,basementsqft,...,censustractandblock,propertylandusedesc,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,storydesc,typeconstructiondesc
37223,10711855,NaN,NaN,2.0,NaN,NaN,NaN,261.0,1087254,NaN,...,6.037113e+13,Single Family Residential,-0.007357,2017-07-07,NaN,NaN,NaN,Central,NaN,NaN
48246,10711877,NaN,NaN,2.0,NaN,NaN,1.0,261.0,1072280,NaN,...,6.037113e+13,Single Family Residential,0.021066,2017-08-29,Central,NaN,NaN,Central,NaN,NaN
15536,10711888,NaN,NaN,2.0,NaN,NaN,1.0,261.0,1340933,NaN,...,6.037113e+13,Single Family Residential,0.077174,2017-04-04,Central,NaN,NaN,Central,NaN,NaN
12106,10711910,NaN,NaN,2.0,NaN,NaN,NaN,261.0,1878109,NaN,...,6.037113e+13,Single Family Residential,-0.041238,2017-03-17,NaN,NaN,NaN,Central,NaN,NaN
13666,10711923,NaN,NaN,2.0,NaN,NaN,NaN,261.0,2190858,NaN,...,6.037113e+13,Single Family Residential,-0.009496,2017-03-24,NaN,NaN,NaN,Central,NaN,NaN


In [10]:
df[df.duplicated(subset='parcelid', keep=False)].head()

,parcelid,typeconstructiontypeid,storytypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,propertylandusetypeid,id,basementsqft,...,censustractandblock,propertylandusedesc,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,storydesc,typeconstructiondesc
9384,10722858,NaN,NaN,2.0,NaN,NaN,NaN,261.0,16179,NaN,...,6.037135e+13,Single Family Residential,0.095171,2017-03-02,NaN,NaN,NaN,Central,NaN,NaN
9385,10722858,NaN,NaN,2.0,NaN,NaN,NaN,261.0,16179,NaN,...,6.037135e+13,Single Family Residential,-0.172843,2017-07-28,NaN,NaN,NaN,Central,NaN,NaN
9306,10732347,NaN,NaN,2.0,NaN,NaN,NaN,261.0,1836115,NaN,...,6.037137e+13,Single Family Residential,0.077198,2017-03-01,NaN,NaN,NaN,Central,NaN,NaN
9307,10732347,NaN,NaN,2.0,NaN,NaN,NaN,261.0,1836115,NaN,...,6.037137e+13,Single Family Residential,-0.221145,2017-07-25,NaN,NaN,NaN,Central,NaN,NaN
1940,10739478,NaN,NaN,2.0,NaN,NaN,1.0,261.0,2119208,NaN,...,6.037800e+13,Single Family Residential,0.083280,2017-01-13,Central,NaN,NaN,Central,NaN,NaN


In [11]:
df = df.drop_duplicates(subset='parcelid', keep='last')

In [12]:
df.parcelid.duplicated().sum()

0

In [13]:
df.shape

(52320, 68)

In [14]:
# Replace blank values with NaN
df = df.replace('',np.nan)

In [15]:
# check if long/lat columns have nulls
df.longitude.isnull().sum(), df.latitude.isnull().sum()

(0, 0)

***

### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [16]:
# Describe without scientific notation
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

,parcelid,typeconstructiontypeid,storytypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,propertylandusetypeid,id,basementsqft,...,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror,buildingclassdesc
count,52320,76,47,33850,0,70,13615,52320,52320,47,...,81,52238,52319,52320,52319,52316,2068,52199,52320,0
mean,1.29968e+07,5.97368,7,3.96561,nan,7.1,2.43959,261,1.49691e+06,678.979,...,1,196636,529824,2016,333492,6454.74,14.1011,6.05024e+13,0.0181378,nan
std,3.35092e+06,0.229416,0,2.56266,nan,2.66567,3.84793,0,859433,711.825,...,0,254286,751830,0,570511,8752.48,2.4004,1.86113e+12,0.176903,nan
min,1.07119e+07,4,7,1,nan,2,1,261,349,38,...,1,129,1000,2016,161,49.18,4,6.0371e+13,-4.65542,nan
25%,1.15102e+07,6,7,2,nan,7,1,261,757614,263.5,...,1,77159,194033,2016,76194,2660.98,14,6.0374e+13,-0.0247003,nan
50%,1.25783e+07,6,7,2,nan,7,1,261,1.50013e+06,512,...,1,131905,374006,2016,218079,4650.57,15,6.03762e+13,0.00694008,nan
75%,1.41304e+07,6,7,7,nan,7,1,261,2.24133e+06,809.5,...,1,226453,619354,2016,408777,7379.27,15,6.05904e+13,0.0406021,nan
max,1.67688e+08,6,7,24,nan,21,13,261,2.98227e+06,3560,...,1,9.1649e+06,4.90612e+07,2016,4.89522e+07,586639,99,4.8303e+14,5.263,nan


In [17]:
df.shape

(52320, 68)

In [18]:
# df.hist()
# plt.tight_layout();

In [19]:
# for col in df.columns:
#     df[col].hist()

In [20]:
# for col in df.columns:
#     print(f'_-------------- {col} ---------------')
#     print(df[col].value_counts())

In [21]:
#df.info(verbose=True)

***

### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [22]:
# show nulls per column
df.isna().sum()

parcelid                      0
typeconstructiontypeid    52244
storytypeid               52273
heatingorsystemtypeid     18470
buildingclasstypeid       52320
                          ...  
architecturalstyledesc    52250
buildingclassdesc         52320
heatingorsystemdesc       18470
storydesc                 52273
typeconstructiondesc      52244
Length: 68, dtype: int64

In [23]:
# show average nulls per column
df.isna().mean()

parcelid                  0.000000
typeconstructiontypeid    0.998547
storytypeid               0.999102
heatingorsystemtypeid     0.353020
buildingclasstypeid       1.000000
                            ...   
architecturalstyledesc    0.998662
buildingclassdesc         1.000000
heatingorsystemdesc       0.353020
storydesc                 0.999102
typeconstructiondesc      0.998547
Length: 68, dtype: float64

In [24]:
# create dataframe that has column name as first column
col_nulls = pd.DataFrame()
col_nulls['columns_name'] = df.isna().sum().index
col_nulls

,columns_name
0,parcelid
1,typeconstructiontypeid
2,storytypeid
3,heatingorsystemtypeid
4,buildingclasstypeid
...,...
63,architecturalstyledesc
64,buildingclassdesc
65,heatingorsystemdesc
66,storydesc


In [25]:
# create new column that hold the sum of nulls from each column
col_nulls['row_null_count'] = df.isna().sum().values
col_nulls

,columns_name,row_null_count
0,parcelid,0
1,typeconstructiontypeid,52244
2,storytypeid,52273
3,heatingorsystemtypeid,18470
4,buildingclasstypeid,52320
...,...,...
63,architecturalstyledesc,52250
64,buildingclassdesc,52320
65,heatingorsystemdesc,18470
66,storydesc,52273


In [26]:
# create new column that hold the average of nulls from each column
col_nulls['row_null_percent'] = df.isna().mean().values
col_nulls

,columns_name,row_null_count,row_null_percent
0,parcelid,0,0.000000
1,typeconstructiontypeid,52244,0.998547
2,storytypeid,52273,0.999102
3,heatingorsystemtypeid,18470,0.353020
4,buildingclasstypeid,52320,1.000000
...,...,...,...
63,architecturalstyledesc,52250,0.998662
64,buildingclassdesc,52320,1.000000
65,heatingorsystemdesc,18470,0.353020
66,storydesc,52273,0.999102


In [27]:
# sort values by percent
col_nulls = col_null.sort_values(by=['row_null_percent'], ascending=False)
col_nulls

,columns_name,row_null_count,row_null_percent
19,finishedsquarefeet15,52320,1.000000
4,buildingclasstypeid,52320,1.000000
64,buildingclassdesc,52320,1.000000
18,finishedsquarefeet13,52320,1.000000
2,storytypeid,52273,0.999102
...,...,...,...
11,bedroomcnt,0,0.000000
29,longitude,0,0.000000
28,latitude,0,0.000000
22,fips,0,0.000000


1. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [29]:
# Ravinder Exmaple
df2 = pd.DataFrame(df.isnull().sum(axis =1), columns = ['num_cols_missing']).reset_index().groupby('num_cols_missing')\
.count().reset_index().rename(columns = {'index': 'num_rows'})

df2['pct_cols_missing'] = df2.num_cols_missing/df.shape[1]
df2.head()

,num_cols_missing,num_rows,pct_cols_missing
0,23,2,0.338235
1,24,12,0.352941
2,25,11,0.367647
3,26,30,0.382353
4,27,177,0.397059


In [30]:
# shows # of null col per row
df.isnull().sum(axis=1)

37223    31
48246    29
15536    31
12106    31
13666    31
         ..
42306    35
31148    36
23911    36
9001     48
28113    48
Length: 52320, dtype: int64

In [31]:
# shows percent of null cols per row
df.isnull().mean(axis=1)

37223    0.455882
48246    0.426471
15536    0.455882
12106    0.455882
13666    0.455882
           ...   
42306    0.514706
31148    0.529412
23911    0.529412
9001     0.705882
28113    0.705882
Length: 52320, dtype: float64

In [32]:
# show # of rows with specific number of null cols
df.isna().sum(axis=1).value_counts()

33    11967
34    11158
32     8885
31     5989
36     4138
35     3469
29     2527
30     2199
37     1020
28      390
38      223
27      177
44       46
26       30
39       29
40       15
24       12
25       11
43       10
42        7
45        6
41        6
23        2
48        2
46        1
47        1
dtype: int64

In [101]:
# shows column nulls in each row
dft = pd.DataFrame(df.isnull().sum(axis=1), columns=['col_nulls'])
dft.head()

,col_nulls
0,36
1,33
2,34
3,32
4,29


In [102]:
# shows index as id
dft = row_nulls.reset_index()
dft.head()

,level_0,index,col_nulls
0,0,0,36
1,1,1,33
2,2,2,34
3,3,3,32
4,4,4,29


In [103]:
# sbow # rows that have a specific number of null columns
dft.col_nulls.value_counts()

33    11995
34    11180
32     8901
31     6004
36     4147
35     3478
29     2531
30     2204
37     1023
28      392
38      227
27      177
44       47
26       30
39       29
40       15
24       12
25       11
43       10
42        7
45        6
41        6
23        2
48        2
46        1
47        1
Name: col_nulls, dtype: int64

In [104]:
# groupby 'null count' to display # of rows have the same # of null cols
dft = dft.groupby('col_nulls').count()
dft.head()

,level_0,index
col_nulls,,
23,2,2
24,12,12
25,11,11
26,30,30
27,177,177


In [105]:
dft_nulls  = dft.treset_index()
dft.head()

AttributeError: 'DataFrame' object has no attribute 'treset_index'

In [106]:
dft.rename(columns = {'index': 'num_row_missing_cols' }, inplace=True)
dft.head()

,level_0,num_row_missing_cols
col_nulls,,
23,2,2
24,12,12
25,11,11
26,30,30
27,177,177


In [107]:
dft['col_null_percent'] = df.isnull().mean(axis=1)
dft.head()

,level_0,num_row_missing_cols,col_null_percent
col_nulls,,,
23,2,2,0.470588
24,12,12,0.485294
25,11,11,0.426471
26,30,30,0.455882
27,177,177,0.455882


In [41]:
# Create df with number of rows with a specific number of null columns
row_nulls = pd.DataFrame(df.isna().sum(axis=1).value_counts(), columns=['num_rows_with_n_null_cols'])
row_nulls.head()

,num_rows_with_n_null_cols
33,11967
34,11158
32,8885
31,5989
36,4138


In [42]:
# make first columnb the number of nulls
row_nulls = dft2.reset_index()
row_nulls.head()

,index,num_rows_with_n_null_cols
0,33,11967
1,34,11158
2,32,8885
3,31,5989
4,36,4138


In [43]:
# rename index to match values
row_nulls = dft2.rename(columns={'index':'n_null_cols'})
row_nulls.head()

,n_null_cols,num_rows_with_n_null_cols
0,33,11967
1,34,11158
2,32,8885
3,31,5989
4,36,4138


In [44]:
# create columsn for percent of null cols
row_nulls['percent_null_cols'] = row_nulls.n_null_cols / df.shape[1]
row_nulls.head()

,n_null_cols,num_rows_with_n_null_cols,percent_null_cols
0,33,11967,0.485294
1,34,11158,0.500000
2,32,8885,0.470588
3,31,5989,0.455882
4,36,4138,0.529412


In [45]:
# sort df by percentn of null cols
row_nulls = row_nulls.sort_values(by=['percent_null_cols'], ascending=False)
row_nulls

,n_null_cols,num_rows_with_n_null_cols,percent_null_cols
23,48,2,0.705882
25,47,1,0.691176
24,46,1,0.676471
20,45,6,0.661765
12,44,46,0.647059
18,43,10,0.632353
19,42,7,0.617647
21,41,6,0.602941
15,40,15,0.588235
14,39,29,0.573529


***

## Prepare

### 1a Remove any properties that are likely to be something other than single unit properties. 

In [46]:
# show value counts for each column, does not include nulls
for col in df.columns:
    print(f'-------------{col}-----------------')
    print(df[df[col].notna()][col].value_counts())

-------------parcelid-----------------
10913809    1
14004381    1
12888190    1
14243968    1
11227265    1
           ..
14147054    1
13096431    1
12660208    1
12789233    1
12853244    1
Name: parcelid, Length: 52320, dtype: int64
-------------typeconstructiontypeid-----------------
6.0    75
4.0     1
Name: typeconstructiontypeid, dtype: int64
-------------storytypeid-----------------
7.0    47
Name: storytypeid, dtype: int64
-------------heatingorsystemtypeid-----------------
2.0     20689
7.0     12527
6.0       517
20.0       85
13.0       16
1.0         7
18.0        6
10.0        2
24.0        1
Name: heatingorsystemtypeid, dtype: int64
-------------buildingclasstypeid-----------------
Series([], Name: buildingclasstypeid, dtype: int64)
-------------architecturalstyletypeid-----------------
7.0     62
3.0      3
2.0      2
21.0     2
8.0      1
Name: architecturalstyletypeid, dtype: int64
-------------airconditioningtypeid-----------------
1.0     11873
13.0     1567
5.0   

Name: typeconstructiondesc, dtype: int64


In [47]:
df.shape

(52320, 68)

In [48]:
df.unitcnt.isna().sum()

18558

In [49]:
# remove propeties that have no bedrooms and no bathrooms and too small of an area
df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0) & (df.unitcnt <= 1) | df.unitcnt.isna() & (df.calculatedfinishedsquarefeet > 500)]

In [50]:
df.shape

(52147, 68)

***

### Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row)

In [51]:
# Ravinder's Example
# def handle_missing_values(df, prop_required_column = .5, prop_required_row = .70):
#     threshold = int(round(prop_required_column*len(df.index),0))
#     df.dropna(axis=1, thresh=threshold, inplace=True)
#     threshold = int(round(prop_required_row*len(df.columns),0))
#     df.dropna(axis=0, thresh=threshold, inplace=True)
#     return df

In [52]:
df.index

Int64Index([37223, 48246, 15536, 12106, 13666,  4381, 36294, 10462, 42662,
             7623,
            ...
            50786,  6576,  8832,  6809, 49219, 37075,  2794, 42306, 31148,
            23911],
           dtype='int64', length=52147)

In [53]:
df.shape

(52147, 68)

In [54]:
len(df.index)

52147

In [55]:
len(df.columns)

68

In [56]:
dft = df.copy()
dft.shape

(52147, 68)

In [57]:
thresh_col = .5 * df.shape[0]
thresh_col

26073.5

In [58]:
dft = dft.dropna(axis=1, thresh=thresh_col)
dft.shape

(52147, 34)

In [59]:
thresh_row = .7 * dft.shape[1]
thresh_row

23.799999999999997

In [60]:
dft.dropna(axis=0, thresh=thresh_row).shape

(52147, 34)

In [61]:
def handle_missing_values(df, percent_required_cols = .5, percent_required_rows = .7):
    
    # set threshold for min of values in columns for dropping
    thresh_col = int(round(percent_required_cols * df.shape[0]))
    
    # drop columns that don't meed threshhold for non-null values (rows without nulls)
    df = df.dropna(axis=1, thresh=thresh_col)
    
    # set threshold for min non-null values for rows (cols without nulls)
    thresh_row = int(round(percent_required_rows * df.shape[1]))
    
    # drop rows with don't meet threshold for non-null values for columns
    df = df.dropna(axis=0, thresh=thresh_row)
    
    return df, thresh_col, thresh_rowresh_row

In [62]:
df, thresh_col, thresh_row  = handle_missing_values(df, percent_required_cols = .5, percent_required_rows = .5)

In [63]:
df.shape, thresh_col, thresh_row

((52147, 34), 26074, 17)

***

### Deal with the rest of the Nulls

In [64]:
df.shape

(52147, 34)

In [65]:
df.isna().sum()

parcelid                            0
heatingorsystemtypeid           18334
propertylandusetypeid               0
id                                  0
bathroomcnt                         0
bedroomcnt                          0
buildingqualitytypeid           18536
calculatedbathnbr                  38
calculatedfinishedsquarefeet        0
finishedsquarefeet12              159
fips                                0
fullbathcnt                        38
latitude                            0
longitude                           0
lotsizesquarefeet                 350
propertycountylandusecode           0
propertyzoningdesc              18471
rawcensustractandblock              0
regionidcity                     1030
regionidcounty                      0
regionidzip                        23
roomcnt                             0
unitcnt                         18456
yearbuilt                          37
structuretaxvaluedollarcnt         69
taxvaluedollarcnt                   1
assessmentye

In [66]:
# remove columns that are not useful
df = df.drop(columns=['parcelid',
                'id',
                 #Description of the allowed land uses (zoning) for that property
                 'propertyzoningdesc', 
                 # Finished living area
                 'finishedsquarefeet12',
                 #  Census tract and block ID combined - also contains blockgroup assignment by extension
                 'censustractandblock',
                      # Type of land use the property is zoned for
                      'propertylandusetypeid',
                      #  Type of home heating system
                      'heatingorsystemtypeid',
                      
                 
                ])

In [67]:
df.shape

(52147, 27)

In [68]:
df.isna().sum()

bathroomcnt                         0
bedroomcnt                          0
buildingqualitytypeid           18536
calculatedbathnbr                  38
calculatedfinishedsquarefeet        0
fips                                0
fullbathcnt                        38
latitude                            0
longitude                           0
lotsizesquarefeet                 350
propertycountylandusecode           0
rawcensustractandblock              0
regionidcity                     1030
regionidcounty                      0
regionidzip                        23
roomcnt                             0
unitcnt                         18456
yearbuilt                          37
structuretaxvaluedollarcnt         69
taxvaluedollarcnt                   1
assessmentyear                      0
landtaxvaluedollarcnt               1
taxamount                           4
propertylandusedesc                 0
logerror                            0
transactiondate                     0
heatingorsys

In [69]:
df.heatingorsystemdesc.value_counts()

Central       20671
Floor/Wall    12508
Forced air      517
Solar            85
None             16
Baseboard         7
Radiant           6
Gravity           2
Yes               1
Name: heatingorsystemdesc, dtype: int64

In [70]:
df[df.fips == 6037].heatingorsystemdesc.value_counts(dropna=False)

Central       20671
Floor/Wall    12479
NaN             503
Solar            85
Name: heatingorsystemdesc, dtype: int64

In [71]:
df[df.fips == 6059].heatingorsystemdesc.value_counts(dropna=False)

NaN           13469
Forced air      517
Floor/Wall       29
None             16
Baseboard         7
Radiant           6
Gravity           2
Yes               1
Name: heatingorsystemdesc, dtype: int64

In [72]:
df[df.fips == 6111].heatingorsystemdesc.value_counts(dropna=False)

NaN    4362
Name: heatingorsystemdesc, dtype: int64

In [73]:
# relacing nulls with 'None'
df.heatingorsystemdesc.fillna('None', inplace=True)

In [74]:
df.isna().sum()

bathroomcnt                         0
bedroomcnt                          0
buildingqualitytypeid           18536
calculatedbathnbr                  38
calculatedfinishedsquarefeet        0
fips                                0
fullbathcnt                        38
latitude                            0
longitude                           0
lotsizesquarefeet                 350
propertycountylandusecode           0
rawcensustractandblock              0
regionidcity                     1030
regionidcounty                      0
regionidzip                        23
roomcnt                             0
unitcnt                         18456
yearbuilt                          37
structuretaxvaluedollarcnt         69
taxvaluedollarcnt                   1
assessmentyear                      0
landtaxvaluedollarcnt               1
taxamount                           4
propertylandusedesc                 0
logerror                            0
transactiondate                     0
heatingorsys

In [77]:
df.buildingqualitytypeid.value_counts(dropna=False)

NaN     18536
6.0     10250
8.0      8219
4.0      8126
7.0      3420
5.0      1497
9.0      1118
11.0      514
10.0      229
3.0       147
12.0       80
1.0         8
2.0         3
Name: buildingqualitytypeid, dtype: int64

In [78]:
df[df.fips == 6037].buildingqualitytypeid.value_counts(dropna=False)

6.0     10250
8.0      8219
4.0      8126
7.0      3420
5.0      1497
9.0      1118
11.0      514
10.0      229
3.0       147
12.0       80
1.0         8
2.0         3
Name: buildingqualitytypeid, dtype: int64

In [79]:
df[df.fips == 6059].buildingqualitytypeid.value_counts(dropna=False)

NaN    14047
Name: buildingqualitytypeid, dtype: int64

In [80]:
df[df.fips == 6111].buildingqualitytypeid.value_counts(dropna=False)

NaN    4362
Name: buildingqualitytypeid, dtype: int64

In [81]:
# droping buildingqualitytypeid because they are null for Ventura and Orange
df.drop(columns=['buildingqualitytypeid'], inplace=True)

In [82]:
df.isna().sum()

bathroomcnt                         0
bedroomcnt                          0
calculatedbathnbr                  38
calculatedfinishedsquarefeet        0
fips                                0
fullbathcnt                        38
latitude                            0
longitude                           0
lotsizesquarefeet                 350
propertycountylandusecode           0
rawcensustractandblock              0
regionidcity                     1030
regionidcounty                      0
regionidzip                        23
roomcnt                             0
unitcnt                         18456
yearbuilt                          37
structuretaxvaluedollarcnt         69
taxvaluedollarcnt                   1
assessmentyear                      0
landtaxvaluedollarcnt               1
taxamount                           4
propertylandusedesc                 0
logerror                            0
transactiondate                     0
heatingorsystemdesc                 0
dtype: int64

In [84]:
df.unitcnt.value_counts(dropna=False)

1.0    33691
NaN    18456
Name: unitcnt, dtype: int64

In [85]:
df[df.fips == 6037].unitcnt.value_counts(dropna=False)

1.0    33686
NaN       52
Name: unitcnt, dtype: int64

In [86]:
df[df.fips == 6059].unitcnt.value_counts(dropna=False)

NaN    14042
1.0        5
Name: unitcnt, dtype: int64

In [87]:
df[df.fips == 6111].unitcnt.value_counts(dropna=False)

NaN    4362
Name: unitcnt, dtype: int64

In [88]:
# drop unitcnt, no longer needed
df.drop(columns=['unitcnt'], inplace=True)

In [89]:
df.isna().sum()

bathroomcnt                        0
bedroomcnt                         0
calculatedbathnbr                 38
calculatedfinishedsquarefeet       0
fips                               0
fullbathcnt                       38
latitude                           0
longitude                          0
lotsizesquarefeet                350
propertycountylandusecode          0
rawcensustractandblock             0
regionidcity                    1030
regionidcounty                     0
regionidzip                       23
roomcnt                            0
yearbuilt                         37
structuretaxvaluedollarcnt        69
taxvaluedollarcnt                  1
assessmentyear                     0
landtaxvaluedollarcnt              1
taxamount                          4
propertylandusedesc                0
logerror                           0
transactiondate                    0
heatingorsystemdesc                0
dtype: int64

In [90]:
# dropping the rest of the nulls
df.dropna(inplace=True)

In [91]:
df.isna().sum()

bathroomcnt                     0
bedroomcnt                      0
calculatedbathnbr               0
calculatedfinishedsquarefeet    0
fips                            0
fullbathcnt                     0
latitude                        0
longitude                       0
lotsizesquarefeet               0
propertycountylandusecode       0
rawcensustractandblock          0
regionidcity                    0
regionidcounty                  0
regionidzip                     0
roomcnt                         0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
assessmentyear                  0
landtaxvaluedollarcnt           0
taxamount                       0
propertylandusedesc             0
logerror                        0
transactiondate                 0
heatingorsystemdesc             0
dtype: int64